The goal of this notebook is to compare two populations...one sampled from the NHANES dataset directly and one sampled from the Gaussian distributions which were in turn obtained from the NHANES dataset...hence the goal is to validate the Gaussian distributions we obtained...

First we read the df with the person information as obtained from the Gaussian distributions, then we create Person-objects, and then the population object. Finally we compare this population and the NHANESDirectSamplePopulation in baseline characteristics, characteristics over time, and cv events.

In [1]:
import os
import copy
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import multivariate_normal

from microsim.person_factory import microsimToNhanes
from microsim.population_factory import PopulationFactory
from microsim.population import Population
#from microsim.population import (NHANESDirectSamplePopulation, 
#                                 build_people_using_nhanes_for_sampling,
#                                 microsimToNhanes, Population, get_nhanes_population)
from microsim.risk_model_repository import RiskModelRepository
from microsim.gender import NHANESGender
from microsim.smoking_status import SmokingStatus
from microsim.race_ethnicity import NHANESRaceEthnicity
from microsim.education import Education
from microsim.treatment import DefaultTreatmentsType,  TreatmentStrategiesType
from microsim.risk_factor import StaticRiskFactorsType, DynamicRiskFactorsType
from microsim.afib_model import AFibPrevalenceModel
from microsim.pvd_model import PVDPrevalenceModel
from microsim.outcome import Outcome, OutcomeType
from microsim.person import Person
from microsim.population_model_repository import PopulationModelRepository
from microsim.cohort_risk_model_repository import (CohortStaticRiskFactorModelRepository,
                                                   CohortDynamicRiskFactorModelRepository,
                                                   CohortDefaultTreatmentModelRepository)
from microsim.outcome_model_repository import OutcomeModelRepository

microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM"
os.chdir(microsimDir+"/CODE/microsim")

dataDir = microsimDir+"/NOTEBOOKS/DATA"

popSize = 100000

In [ ]:
categoricalVars = ["gender", "smokingStatus", "raceEthnicity", "statin",'education',
                  'alcoholPerWeek','anyPhysicalActivity','antiHypertensiveCount']
continuousVars = ['age', 'hdl', 'bmi', 'totChol', 'trig', 'a1c', 'ldl', 'waist', 'creatinine', 'sbp', 'dbp']

In [ ]:
nhanesDf = pd.read_stata(microsimDir + "/CODE/microsim/microsim/data/fullyImputedDataset.dta")
nhanesDf = nhanesDf.rename(columns={"level_0":"name"})

In [ ]:
df = pd.read_csv(dataDir+"/nhanes-as-gaussians-02.csv")
#df = pd.read_csv(dataDir+"/nhanes-persons-from-uniforms.csv")
df.head()

In [ ]:
df["age"].describe()

In [ ]:
df.shape

In [ ]:
#bring the original nhanes weights because these will help me do the sampling
df = df.merge(nhanesDf[["name","WTINT2YR"]], on="name", how="inner").copy()
df.shape

In [ ]:
def build_person(x, initializationModelRepository):
    """Takes all Person-instance-related data via x and initializationModelRepository and organizes it,
       passes the organized data to the Person class and returns a Person instance."""
    
    rng = np.random.default_rng()
        
    name = x.name
    
    personStaticRiskFactors = {
                        StaticRiskFactorsType.RACE_ETHNICITY.value: NHANESRaceEthnicity(int(x.raceEthnicity)),
                        StaticRiskFactorsType.EDUCATION.value: Education(int(x.education)),
                        StaticRiskFactorsType.GENDER.value: NHANESGender(int(x.gender)),
                        StaticRiskFactorsType.SMOKING_STATUS.value: SmokingStatus(int(x.smokingStatus))}
        
    #use this to get the bounds imposed on the risk factors in a bit
    rfRepository = RiskModelRepository()
        
    #TO DO: find a way to include everything here, including the rfs that need initialization
    #the PVD model would be easy to implement, eg with an estimate_next_risk_for_patient_characteristics function
    #but the AFIB model would be more difficult because it relies on statsmodel_logistic_risk file
    #for now include None, in order to create the risk factor lists correctly at the Person instance
    personDynamicRiskFactors = dict()
    for rfd in DynamicRiskFactorsType:
        #if rfd==DynamicRiskFactorsType.ALCOHOL_PER_WEEK:
        #    personDynamicRiskFactors[rfd.value] = AlcoholCategory.get_category_for_consumption(rfRepository.apply_bounds(rfd.value, x[microsimToNhanes[rfd.value]]))
        #else:
        if (rfd!=DynamicRiskFactorsType.PVD) & (rfd!=DynamicRiskFactorsType.AFIB):
            personDynamicRiskFactors[rfd.value] = rfRepository.apply_bounds(rfd.value, x[rfd.value])
    personDynamicRiskFactors[DynamicRiskFactorsType.AFIB.value] = None
    personDynamicRiskFactors[DynamicRiskFactorsType.PVD.value] = None

    #Q: do we need otherLipid treatment? I am not bringing it to the Person objects for now.
    #A: it is ok to leave it out as we do not have a model to update this. It is also very rarely taking place in the population anyway.
    #also: used to have round(x.statin) but NHANES includes statin=2...
    personDefaultTreatments = {
                        DefaultTreatmentsType.STATIN.value: bool(x.statin),
                        #DefaultTreatmentsType.OTHER_LIPID_LOWERING_MEDICATION_COUNT.value: x.otherLipidLowering,
                        DefaultTreatmentsType.ANTI_HYPERTENSIVE_COUNT.value: x.antiHypertensiveCount}

    personTreatmentStrategies = dict(zip([strategy.value for strategy in TreatmentStrategiesType],
                                         #[None for strategy in range(len(TreatmentStrategiesType))]))
                                         [{"status": None} for strategy in range(len(TreatmentStrategiesType))]))

    personOutcomes = dict(zip([outcome for outcome in OutcomeType],
                                  [list() for outcome in range(len(OutcomeType))]))
    #add pre-simulation stroke outcomes
    #selfReportStrokeAge=x.selfReportStrokeAge
    #Q: we should not add the stroke outcome in case of "else"?
    #if selfReportStrokeAge is not None and selfReportStrokeAge > 1:
    #        selfReportStrokeAge = selfReportStrokeAge if selfReportStrokeAge <= x.age else x.age
    #        personOutcomes[OutcomeType.STROKE].append((selfReportStrokeAge, StrokeOutcome(False, None, None, None, priorToSim=True)))
    #add pre-simulation mi outcomes
    #selfReportMIAge=rng.integers(18, x.age) if x.selfReportMIAge == 99999 else x.selfReportMIAge
    #if selfReportMIAge is not None and selfReportMIAge > 1:
    #        selfReportMIAge = selfReportMIAge if selfReportMIAge <= x.age else x.age
    #        personOutcomes[OutcomeType.MI].append((selfReportMIAge, Outcome(OutcomeType.MI, False, priorToSim=True)))

    person = Person(name,
                   personStaticRiskFactors,
                   personDynamicRiskFactors,
                   personDefaultTreatments,
                   personTreatmentStrategies,
                   personOutcomes)

    #TO DO: find a way to initialize these rfs above with everything else
    person._pvd = [initializationModelRepository[DynamicRiskFactorsType.PVD].estimate_next_risk(person)]
    person._afib = [initializationModelRepository[DynamicRiskFactorsType.AFIB].estimate_next_risk(person)]
    return person


In [ ]:
initializationModelRepository = {DynamicRiskFactorsType.AFIB: AFibPrevalenceModel(), 
                                     DynamicRiskFactorsType.PVD: PVDPrevalenceModel()}

In [ ]:
dfSample = df.sample(popSize, weights=df.WTINT2YR, replace=True)

In [ ]:
people = dfSample.apply(lambda x: build_person(x, initializationModelRepository), axis=1)

In [ ]:
#sets the unique identifier for each Person instance
noneList = list(map(lambda person, i: setattr(person, "_index", i), people, range(len(people)))) 

In [ ]:
popModelRepository = PopulationModelRepository(CohortDynamicRiskFactorModelRepository(),
                                                           CohortDefaultTreatmentModelRepository(),
                                                           OutcomeModelRepository(),
                                                           CohortStaticRiskFactorModelRepository()) 
pop = Population(people, popModelRepository)

In [ ]:
pop._people.shape

In [2]:
#pop2 = get_nhanes_population(1999)
#pop2 = NHANESDirectSamplePopulation(popSize, 1999)
pop = PopulationFactory.get_nhanes_population(n=popSize, year=1999, nhanesWeights=True, distributions=True)
pop.advance(1)
pop2 = PopulationFactory.get_nhanes_population(n=popSize, year=1999, nhanesWeights=True)
pop2.advance(1)
#pop2.print_baseline_summary()

In [3]:
pop.print_baseline_summary_comparison(pop2)

                                 self                                             other
                                 min    0.25   med    0.75    max   mean    sd    min    0.25   med    0.75    max   mean    sd
                           age   18.0   32.0   42.0   54.0   93.0   44.1   15.3   18.0   30.0   41.0   55.0   85.0   43.9   17.2
                           sbp   77.7  112.1  122.0  132.5  221.7  123.0   16.5   72.7  109.3  119.3  131.3  231.3  122.6   18.9
                           dbp   38.5   65.2   72.1   78.8  106.8   72.2   10.1   40.0   65.3   72.0   79.3  132.0   72.5   11.1
                           a1c    3.7    5.0    5.2    5.6   10.6    5.4    0.7    2.5    5.0    5.2    5.5   15.1    5.4    0.9
                           hdl   17.5   42.5   51.1   60.5  113.0   52.3   13.4    8.0   40.0   48.0   59.0  151.0   50.8   15.7
                           ldl   40.9  108.1  127.4  148.0  257.5  128.0   30.0   28.0   98.0  120.0  145.0  354.0  123.0   36.6
          

In [4]:
popages = pop.get_age_of_all_years_in_sim()
popageCounts = pop.get_age_counts(popages)
popages2 = pop2.get_age_of_all_years_in_sim()
popageCount2s = pop2.get_age_counts(popages2)

print("age pop-count pop2-count %diff") 
for key in sorted(popageCounts):
    if key in popageCount2s:
        pcDiff =  (popageCounts[key] - popageCount2s[key])/popageCount2s[key]
        print(f"{key} {popageCounts[key]:>6.0f} {popageCount2s[key]:>10.0f} {pcDiff:>9.2f}")
    else:
        print(f"{key} {popageCounts[key]:>6.0f}")

age pop-count pop2-count %diff
18   1443       1792     -0.19
19    677       2020     -0.66
20   1017       1906     -0.47
21   1351       2224     -0.39
22   1409       2443     -0.42
23   1719       1946     -0.12
24   1614       1692     -0.05
25   1675       1583      0.06
26   1802       1975     -0.09
27   1769       1552      0.14
28   2138       2032      0.05
29   2193       2258     -0.03
30   2041       1990      0.03
31   2151       1994      0.08
32   1981       2492     -0.21
33   2234       2080      0.07
34   2434       2035      0.20
35   2511       2528     -0.01
36   2507       2039      0.23
37   2388       2612     -0.09
38   2787       2207      0.26
39   2435       2799     -0.13
40   2203       2292     -0.04
41   2912       2083      0.40
42   2536       2003      0.27
43   2387       1780      0.34
44   2120       2135     -0.01
45   2393       2114      0.13
46   2415       2121      0.14
47   2464       1760      0.40
48   1746       1537      0.14
49   235

In [5]:
#personsOut = list(filter( lambda x: ((x._sbp[0]>=72.) & (x._sbp[0]<=81.)) | ((x._sbp[0]>=207.) & (x._sbp[0]<=266.)) , pop2._people))
#len(personsOut)

In [6]:
#personsOutUnique = set(map( lambda x: x._name, personsOut) )
#len(personsOutUnique)

In [7]:
%%time
pop.advance(17, None, nWorkers=5)

CPU times: user 36.1 s, sys: 6.27 s, total: 42.4 s
Wall time: 7min 59s


In [8]:
%%time
pop2.advance(17, None, nWorkers=5)

CPU times: user 52.1 s, sys: 14.1 s, total: 1min 6s
Wall time: 8min 13s


In [9]:
pop.print_lastyear_summary_comparison(pop2)

                                 self                                             other
                                 min    0.25   med    0.75    max   mean    sd    min    0.25   med    0.75    max   mean    sd
                           age   35.0   48.0   57.0   67.0  100.0   57.8   12.7   35.0   45.0   55.0   65.0  102.0   56.0   13.6
                           sbp   83.6  123.2  131.3  140.2  193.4  132.0   13.2   80.2  120.3  128.6  138.4  253.5  130.1   15.0
                           dbp   47.5   73.7   81.3   88.7  124.8   81.5   10.8   44.3   73.6   81.7   89.9  152.5   81.9   11.6
                           a1c    4.6    5.5    5.8    6.1    8.6    5.8    0.4    4.0    5.5    5.7    6.1   10.6    5.8    0.6
                           hdl    5.4   38.9   51.1   63.7  138.0   52.1   18.6    5.4   35.0   47.2   62.3  171.2   49.8   21.2
                           ldl    8.1   94.2  112.0  129.5  219.0  111.7   26.3    8.1   94.2  114.9  133.2  250.9  112.8   30.4
          

In [10]:
%%time
pop.print_cv_standardized_rates()

standardized rates (per 100,000)    all        black      white
                            mi      216.6      219.5      227.4
                        stroke      153.0      254.8      118.0
                         death      819.7      931.1      841.3
                            cv      369.5      474.3      345.4
                         noncv      770.6      873.2      793.6
                      dementia      258.2      546.7      222.6
CPU times: user 6min 24s, sys: 32.6 s, total: 6min 57s
Wall time: 7min 2s


In [11]:
%%time
pop2.print_cv_standardized_rates()

standardized rates (per 100,000)    all        black      white
                            mi      238.6      216.4      252.0
                        stroke      158.9      290.2      125.0
                         death      864.9      956.9      878.6
                            cv      397.5      506.6      377.0
                         noncv      807.3      888.2      823.9
                      dementia      275.7      577.5      235.1
CPU times: user 6min 34s, sys: 28.5 s, total: 7min 3s
Wall time: 7min 6s
